In [1]:
# 引入需要的库
import pandas as pd
import numpy as np
from scipy.optimize import minimize, Bounds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam



e:\Deep_learning_study\MCM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
# --------------------------
# 模块1：数据预处理与特征工程
# --------------------------
class DataProcessor:
    def __init__(self, pareto_path, tourism_path, tourists_path):
        self.pareto_path = pareto_path
        self.tourism_path = tourism_path
        self.tourists_path = tourists_path
        
    def load_data(self):
        """加载所有数据集并合并"""
        # 读取Pareto前沿数据
        self.pareto_df = pd.read_excel(self.pareto_path)
        
        # 读取每日旅游数据（包含区域信息）
        self.tourism_df = pd.read_excel(self.tourism_path)
        self.tourism_df['date'] = pd.to_datetime(self.tourism_df['date'])
        self.tourism_df = self.tourism_df[self.tourism_df['carbon'] >= 0]  # 清除负碳排放
        
        # 读取月度游客数据
        self.tourists_monthly_df = pd.read_excel(self.tourists_path)
        self.tourists_monthly_df['date'] = pd.to_datetime(self.tourists_monthly_df['date'])
        
        return self
    
    def feature_engineering(self):
        """生成关键特征"""
        # 按区域计算平均碳排放
        region_carbon = self.tourism_df.groupby('region')['carbon'].mean().reset_index()
        region_carbon.columns = ['region', 'region_carbon_avg']
        self.tourism_df = pd.merge(self.tourism_df, region_carbon, on='region', how='left')
        
        # 计算游客数量与碳排放的比值（旅游效率）
        self.tourism_df['efficiency'] = self.tourism_df['tourists'] / (self.tourism_df['carbon'] + 1e-6)
        
        # 合并月度数据到每日数据
        self.tourism_df['month'] = self.tourism_df['date'].dt.to_period('M')
        self.tourists_monthly_df['month'] = self.tourists_monthly_df['date'].dt.to_period('M')
        merged_df = pd.merge(
            self.tourism_df,
            self.tourists_monthly_df[['month', 'tourists']],
            on='month',
            suffixes=('_daily', '_monthly')
        )
        
        return merged_df
    
    def normalize_data(self, df):
        """数据标准化"""
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(df[['Economic_Profit', 'Glacier_Retreat', 'Social_Satisfaction']])
        df[['profit_norm', 'glacier_norm', 'social_norm']] = scaled_data
        return df, scaler

In [3]:
# --------------------------
# 模块2：多目标优化模型
# --------------------------
class TourismOptimizer:
    def __init__(self, data, params):
        """
        :param data: 预处理后的数据
        :param params: 模型参数
            - weights: 目标权重 [经济, 环境, 社会]
            - max_tourists: 最大游客容量
            - carbon_cap: 碳排放上限
        """
        self.data = data
        self.weights = params['weights']
        self.max_tourists = params['max_tourists']
        self.carbon_cap = params['carbon_cap']
        
    def objective(self, x):
        """多目标优化函数"""
        # x = [游客数量, 碳排放量, 基础设施投资比例]
        profit = x[0] * 5000  # 假设每位游客贡献$5000经济收益
        carbon_penalty = np.maximum(x[1] - self.carbon_cap, 0) * 1e6  # 超碳排放罚款
        social_score = 1 / (1 + np.exp(-x[2]))  # 基础设施投资对社会满意度的影响
        
        # 加权目标（最大化经济和社会，最小化环境）
        return -(self.weights[0]*profit - self.weights[1]*carbon_penalty + self.weights[2]*social_score)
    
    def constraints(self):
        """动态约束条件"""
        cons = [
            {'type': 'ineq', 'fun': lambda x: self.max_tourists - x[0]},  # 游客上限
            {'type': 'ineq', 'fun': lambda x: self.carbon_cap - x[1]},    # 碳排放上限
            {'type': 'ineq', 'fun': lambda x: x[2]}                       # 投资比例非负
        ]
        return cons
    
    def optimize(self):
        """执行优化"""
        bounds = Bounds(
            [0, 0, 0],                    # 下限
            [self.max_tourists, np.inf, 1] # 上限
        )
        
        res = minimize(
            self.objective,
            x0=[self.max_tourists/2, self.carbon_cap/2, 0.5],
            method='SLSQP',
            bounds=bounds,
            constraints=self.constraints()
        )
        return res

In [4]:
# --------------------------
# 模块3：加入时间序列预测
# --------------------------


class TimeSeriesPredictor:
    def __init__(self, data, target_column='tourists', date_column='date'):
        self.data = data.sort_values(date_column)
        self.target = target_column
        self.date_col = date_column
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        
    def preprocess(self, forecast_days=30):
        """数据预处理与特征工程"""
        # 生成时序特征
        df = self.data.set_index(self.date_col)[self.target].resample('D').mean().ffill()
        df = pd.DataFrame(df)
        df['day_of_week'] = df.index.dayofweek
        df['day_of_month'] = df.index.day
        df['month'] = df.index.month
        return df
    
    def prophet_predict(self, periods=365):
        """使用Facebook Prophet进行预测"""
        df = self.data[[self.date_col, self.target]].rename(columns={self.date_col: 'ds', self.target: 'y'})

        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05
        )
        model.add_country_holidays(country_name='US')
        model.fit(df)
        
        future = model.make_future_dataframe(periods=periods)
        forecast = model.predict(future)
        return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    
    def lstm_predict(self, look_back=15, forecast_days=30):
        """使用LSTM神经网络进行预测"""
        dataset = self.data[self.target].values.reshape(-1, 1)
        dataset = self.scaler.fit_transform(dataset)
        
        # 在数据集创建之前，添加调试信息
        print(f"Dataset length: {len(dataset)}")
        print(f"Look back: {look_back}, Forecast days: {forecast_days}")
        
        # 创建时序数据集
        X, Y = [], []
        for i in range(len(dataset) - look_back - forecast_days):
            X.append(dataset[i:(i + look_back), 0])
            Y.append(dataset[i + look_back:i + look_back + forecast_days, 0])
        
        # 检查是否有足够的数据点
        if len(X) == 0 or len(Y) == 0:
            raise ValueError("Not enough data to create the specified look_back and forecast_days sequences.")
        
        X = np.array(X)
        Y = np.array(Y)
        
        # 构建LSTM模型
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
        model.add(LSTM(50))
        model.add(Dense(forecast_days))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        
        # 训练模型
        model.fit(X.reshape(X.shape[0], X.shape[1], 1), Y, 
                epochs=50, batch_size=32, verbose=0)
        
        # 生成预测
        last_sequence = dataset[-look_back:]
        predictions = model.predict(last_sequence.reshape(1, look_back, 1))
        return self.scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
    def hybrid_predict(self, periods=365):
        """混合预测方法：Prophet + LSTM残差修正"""
        # 第一阶段：Prophet基础预测
        prophet_result = self.prophet_predict(periods)
        
        # 第二阶段：计算残差
        residuals = self.data[self.target] - prophet_result['yhat'][:-periods]
        
        # 确保残差的长度与LSTM预测的长度一致
        lstm_residuals = self.lstm_predict(look_back=15, forecast_days=periods)
        
        # 合并结果
        final = prophet_result.copy()
        # 使用预测周期内的残差进行修正
        final['hybrid'] = final['yhat'][-periods:] + lstm_residuals
        return final


In [5]:
# --------------------------
# 模块4：可视化与报告生成
# --------------------------
class Visualizer:
    @staticmethod
    def plot_pareto_front(df):
        """3D帕累托前沿可视化"""
        fig = plt.figure(figsize=(12, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        xs = df['profit_norm']
        ys = df['glacier_norm']
        zs = df['social_norm']
        
        ax.scatter(xs, ys, zs, c=zs, cmap=cm.viridis, marker='o')
        ax.set_xlabel('Economic Profit')
        ax.set_ylabel('Glacier Retreat')
        ax.set_zlabel('Social Satisfaction')
        plt.title("3D Pareto Front")
        plt.show()
    
    @staticmethod
    def plot_scenario_comparison(results):
        """情景对比雷达图"""
        labels = ['Economic', 'Environment', 'Social']
        angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
        angles += angles[:1]  # 闭合雷达图

        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, polar=True)

        # 经济优先
        economic_values = [
            results['economic']['profit'],
            -results['economic']['carbon'],  # 负值表示最小化目标
            results['economic']['social_score']
        ]
        economic_values += economic_values[:1]  # 闭合雷达图
        ax.plot(angles, economic_values, 'o-', linewidth=2, label='Economic Priority')

        # 环境优先
        environmental_values = [
            results['environmental']['profit'],
            -results['environmental']['carbon'],
            results['environmental']['social_score']
        ]
        environmental_values += environmental_values[:1]
        ax.plot(angles, environmental_values, 'o-', linewidth=2, label='Environmental Priority')

        # 社会优先
        social_values = [
            results['social']['profit'],
            -results['social']['carbon'],
            results['social']['social_score']
        ]
        social_values += social_values[:1]
        ax.plot(angles, social_values, 'o-', linewidth=2, label='Social Priority')

        ax.set_thetagrids(np.degrees(angles[:-1]), labels)
        plt.legend(loc='upper right')
        plt.title("Scenario Comparison")
        plt.show()

In [6]:
# --------------------------
# 模块5: 主程序
# --------------------------
if __name__ == "__main__":
    # 初始化数据处理器
    processor = DataProcessor(
        pareto_path="pareto_data.xlsx",
        tourism_path="tourism_data.xlsx",
        tourists_path="tourists_data.xlsx"
    )
    merged_data = processor.load_data().feature_engineering()
        # ========== 新增时间序列预测模块 ==========
    # 游客数量预测
    tourist_predictor = TimeSeriesPredictor(
        processor.tourists_monthly_df, 
        target_column='tourists',
        date_column='date'
    )
    prophet_forecast = tourist_predictor.prophet_predict(periods=1)  # 预测未来1个月
    lstm_forecast = tourist_predictor.lstm_predict(forecast_days=30)  # 预测未来30天
    
    # 碳排放预测
    carbon_predictor = TimeSeriesPredictor(
        processor.tourism_df,
        target_column='carbon',
        date_column='date'
    )
    carbon_forecast = carbon_predictor.hybrid_predict(periods=30)
    
    # 将预测结果整合到优化模型输入数据
    def integrate_forecasts(original, forecast, freq='D'):
        """融合历史数据与预测数据"""
        forecast_df = forecast.set_index('ds') if 'ds' in forecast.columns else pd.DataFrame(
            index=pd.date_range(start=original.index[-1], periods=len(forecast), freq=freq),
            data=forecast
        )
        return pd.concat([original, forecast_df], axis=0)
    
    # 检验
    print(merged_data.columns)
    
    # 检查 integrate_forecasts 返回的 DataFrame
    lstm_forecast_df = pd.DataFrame(
        lstm_forecast,
        columns=['tourists_monthly']  # 使用与目标列相同的名称
    )

    # 调用 integrate_forecasts 并检查返回值
    integrated_forecast = integrate_forecasts(
        merged_data['tourists_monthly'], 
        lstm_forecast_df, 
        freq='M'
    )

    # 打印 integrated_forecast 的信息
    print("Integrated Forecast DataFrame:")
    print(integrated_forecast.head())  # 打印前几行
    print(integrated_forecast.info())  # 打印结构信息

    # 检查 merged_data['tourists_monthly'] 的内容
    print("\nMerged Data 'tourists_monthly' Column Before Update:")
    print(merged_data['tourists_monthly'].head())  # 打印前几行
    print(merged_data['tourists_monthly'].info())  # 打印结构信息

    # 更新游客数据
    merged_data['tourists_monthly'] = integrated_forecast

    # 再次检查更新后的 merged_data['tourists_monthly']
    print("\nMerged Data 'tourists_monthly' Column After Update:")
    print(merged_data['tourists_monthly'].head())  # 打印前几行
    print(merged_data['tourists_monthly'].info())  # 打印结构信息
    


    # 更新碳排放数据
    merged_data['carbon'] = integrate_forecasts(
        merged_data['carbon'],
        carbon_forecast[['ds', 'hybrid']].rename(columns={'ds': 'date', 'hybrid': 'carbon'}),
        freq='D'
    )
    normalized_data, scaler = processor.normalize_data(processor.pareto_df)
    
    # 情景参数设置
    scenarios = {
        'economic': {'weights': [0.7, 0.1, 0.2], 'max_tourists': 20000, 'carbon_cap': 150},
        'environmental': {'weights': [0.2, 0.7, 0.1], 'max_tourists': 15000, 'carbon_cap': 100},
        'social': {'weights': [0.1, 0.2, 0.7], 'max_tourists': 18000, 'carbon_cap': 120}
    }
    
    # 运行优化
    results = {}
    for scenario, params in scenarios.items():
        optimizer = TourismOptimizer(normalized_data, params)
        res = optimizer.optimize()
        if res.success:
            results[scenario] = {
                'tourists': res.x[0],
                'carbon': res.x[1],
                'investment': res.x[2],
                'profit': res.x[0] * 5000,
                'social_score': 1 / (1 + np.exp(-res.x[2]))
            }
            
    # ========== 新增预测可视化 ==========
    plt.figure(figsize=(12, 6))
    plt.plot(prophet_forecast['ds'], prophet_forecast['yhat'], label='Prophet Forecast')
    plt.plot(lstm_forecast, label='LSTM Forecast', linestyle='--')
    plt.fill_between(prophet_forecast['ds'], 
                    prophet_forecast['yhat_lower'],
                    prophet_forecast['yhat_upper'],
                    alpha=0.2)
    plt.title("Tourist Number Forecast Comparison")
    plt.legend()
    plt.show()
    
    # 可视化
    Visualizer.plot_pareto_front(normalized_data)
    Visualizer.plot_scenario_comparison(results)
    
    # 生成报告
    report = f"""
    === 朱诺市可持续旅游发展建议报告 ===
    
    最优解参数：
    - 经济优先情景：游客={results['economic']['tourists']:.0f}人/天, 
                   碳排放={results['economic']['carbon']:.1f}吨, 
                   社会评分={results['economic']['social_score']:.2f}
    - 环境优先情景：游客={results['environmental']['tourists']:.0f}人/天, 
                   碳排放={results['environmental']['carbon']:.1f}吨,
                   社会评分={results['environmental']['social_score']:.2f}
    - 社会优先情景：游客={results['social']['tourists']:.0f}人/天, 
                   碳排放={results['social']['carbon']:.1f}吨,
                   社会评分={results['social']['social_score']:.2f}
    
    建议措施：
    1. 动态游客管理：在冰川区域实施分时预约制，高峰期限流至{results['environmental']['tourists']}人/天
    2. 绿色交通计划：将雨林区域的接驳车电动化，预计减少碳排放{results['environmental']['carbon']*0.3:.1f}吨/天
    3. 社区基金：将{results['social']['investment']*100:.1f}%的旅游收入用于居民福利
    4. 数字导览系统：投资智能设备降低人工压力，提升游客体验评分至{results['social']['social_score']*10:.1f}/10分
    """
    print(report)

15:26:40 - cmdstanpy - INFO - Chain [1] start processing
15:26:41 - cmdstanpy - INFO - Chain [1] done processing


Dataset length: 48
Look back: 15, Forecast days: 30
1/1 [==============================] - 0s 499ms/step


15:26:45 - cmdstanpy - INFO - Chain [1] start processing
15:26:45 - cmdstanpy - INFO - Chain [1] done processing


Dataset length: 358
Look back: 15, Forecast days: 30
1/1 [==============================] - 1s 522ms/step
Index(['date', 'tourists_daily', 'carbon', 'region', 'region_carbon_avg',
       'efficiency', 'month', 'tourists_monthly'],
      dtype='object')
Integrated Forecast DataFrame:
           0  tourists_monthly
0  33.589463               NaN
1  33.589463               NaN
2  33.589463               NaN
3  33.589463               NaN
4  33.589463               NaN
<class 'pandas.core.frame.DataFrame'>
Index: 388 entries, 0 to 1972-06-30 00:00:00.000000357
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   0                 358 non-null    float64
 1   tourists_monthly  0 non-null      float32
dtypes: float32(1), float64(1)
memory usage: 7.6+ KB
None

Merged Data 'tourists_monthly' Column Before Update:
0    33.589463
1    33.589463
2    33.589463
3    33.589463
4    33.589463
Name: tourists_monthly, dtype:

ValueError: Columns must be same length as key

In [ ]:
# 动态约束管理
def constraints(self):
    return [
        {'type': 'ineq', 'fun': lambda x: self.max_tourists - x[0]},
        {'type': 'ineq', 'fun': lambda x: self.carbon_cap - x[1]},
        {'type': 'ineq', 'fun': lambda x: x[2]}
    ]

In [ ]:
# 非线性目标函数
def objective(self, x):
    profit = x[0] * 5000  # 线性经济收益
    carbon_penalty = np.maximum(x[1] - self.carbon_cap, 0) * 1e6  # 分段惩罚函数
    social_score = 1 / (1 + np.exp(-x[2]))  # Sigmoid函数映射
    return -(self.weights[0]*profit - self.weights[1]*carbon_penalty + self.weights[2]*social_score)